# Simulation on synthetic data (Table 1)

This notebook contains the code form the simulation on the synthetic data

In [ ]:
%load_ext autoreload
%autoreload 2

# general imports
import numpy as np
import sys, random
import os, warnings
import itertools as iter
from copy import deepcopy
from tqdm.notebook import tqdm
from scipy.special import expit
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

# initialization
rng = np.random.default_rng(1234)
random.seed()

# add to path
print(os.getcwd())
sys.path.append(os.getcwd())

# import ai360
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, CompasDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas

# import main code
from utils import *
import algorithms as denoisedfair
from lamy_noise_fairlearn.util import *
from awasthi_equalized_odds_under_perturbation.equalized_odds import *

from scipy.optimize import minimize

In [ ]:
exec(open('baseSimulation.py').read())

## Matplotlib parameters

In [ ]:
# RC parameters
from matplotlib import rc, rcParams

rcParams.update({
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
        'figure.figsize': (10,6),
})

import datetime
import string

home_folder = '.'

def file_str():
    """ Auto-generates file name."""
    now = datetime.datetime.now()
    return now.strftime("H%HM%MS%S_%m-%d-%y")

rand_string = lambda length: ''.join(random.choice(string.ascii_lowercase) for i in range(length))

def pdf_savefig():
    """ Saves figures as pdf """
    fname = file_str()+rand_string(20)
    plt.savefig(home_folder+f"/figs/{fname}.pdf")

def eps_savefig():
    """ Saves figure as encapsulated postscript file (vector format)
        so that it isn't pixelated when we put it into a pdf. """
    pdf_savefig()

## Functions to generate adversarial noise

In [ ]:
flipping_far_from_boundary = lambda feature_names, features, labels,\
                                        name, eta0, eta1: \
                                                    flipping_syn_far_from_boundary(feature_names,\
                                                    features, labels, name, eta0, eta1, pred_lab=0,
                                                    rng_loc=None, in_use_prot_attr = False)

##  Generate synthetic data

In [ ]:
def gen_syn_data(n):
    data = []
    labels = []
    zer = np.random.normal(0,0.2,20*n)
    one = np.random.normal(1,0.2,20*n)
    #
    label = np.random.rand(20*n)
    #
    z=0
    o=0
    #
    sft = -0.5 # shift (to center at 0)
    # generate samples for z=1
    for i in range(int(0.333*n//4)): data.append([1, zer[o], 1+one[o+1]+sft]); labels.append([1]); o+=2
    for i in range(int(0.555*n//4)): data.append([1, zer[o], one[o+1]+sft]); labels.append([1]); o+=2
    for i in range(int(0.222*n//4)): data.append([1, zer[o], zer[o+1]+sft]); labels.append([0]); o+=2
    for i in range(int(0.888*n//4)): data.append([1, zer[o], -one[o+1]+sft]); labels.append([0]); o+=2
    # generate samples for z=2
    for i in range(int(0.888*n//4)): data.append([0, zer[o], 1+one[o+1]+sft]); labels.append([1]); o+=2
    for i in range(int(0.222*n//4)): data.append([0, zer[o], one[o+1]+sft]); labels.append([1]); o+=2
    for i in range(int(0.555*n//4)): data.append([0, zer[o], zer[o+1]+sft]); labels.append([0]); o+=2
    for i in range(int(0.333*n//4)): data.append([0, zer[o], -one[o+1]+sft]); labels.append([0]); o+=2
    return np.array(data), np.array(labels)

In [ ]:
os.system('mkdir figs')

In [ ]:
N = 6000
syn_data, syn_labels = gen_syn_data(N)

indMpos, indMneg, indFpos, indFneg = [], [], [], []

for i in range(len(syn_labels)):
    if syn_data[i][0] == 0 and syn_labels[i] == 1: indMpos.append(i)
    if syn_data[i][0] == 0 and syn_labels[i] == 0: indMneg.append(i)
    if syn_data[i][0] == 1 and syn_labels[i] == 1: indFpos.append(i)
    if syn_data[i][0] == 1 and syn_labels[i] == 0: indFneg.append(i)
        
        
##############################################################################
########################## Plot for Z=1 ######################################
##############################################################################
plt.scatter(syn_data[:,1][indMpos], syn_data[:,2][indMpos], marker='+', alpha=0.5,\
            color='blue', label='Positive samples', linewidth=2)
plt.scatter(syn_data[:,1][indMneg], syn_data[:,2][indMneg], marker='_', alpha=0.5,\
            color='red', label='Negative samples', linewidth=2)

plt.ylim(-2,2)
plt.xlim(-1,1)
plt.xlabel('Feature 2', fontsize=25)
plt.ylabel('Feature 1', fontsize=25)
plt.tick_params(axis='both', which='major', labelsize=20)

legend = plt.legend(shadow=False, fontsize=20, bbox_to_anchor=(0.51, 0.3, 0.9, .102))

plt.title(f'Synthetic dataset: Samples with $Z=1$', fontsize=25)


pdf_savefig()
plt.show()


##############################################################################
########################## Plot for Z=2 ######################################
##############################################################################
plt.scatter(syn_data[:,1][indFpos], syn_data[:,2][indFpos], marker='+', alpha=0.5,\
            color='blue', label='Positive samples', linewidth=2)
plt.scatter(syn_data[:,1][indFneg], syn_data[:,2][indFneg], marker='_', alpha=0.5,\
            color='red', label='Negative samples', linewidth=2)

plt.ylim(-2,2)
plt.xlim(-1,1)
plt.xlabel('Feature 2', fontsize=25)
plt.ylabel('Feature 1', fontsize=25)
plt.tick_params(axis='both', which='major', labelsize=20)

legend = plt.legend(shadow=False, fontsize=20, bbox_to_anchor=(0.51, 0.3, 0.9, .102))

plt.title(f'Synthetic dataset: Samples of with $Z=2$', fontsize=25)

pdf_savefig()
plt.show()

## Simulation code

In [ ]:
verbose = False
ITERS = 100
CORES = 10

In [ ]:
def gen_data(eta=[0,0],flip_func=flipping_far_from_boundary, N=1000):
    syn_data, syn_labels = gen_syn_data(int(N*0.7))
    syn_data_test, syn_labels_test = gen_syn_data(int(N*0.3))
    
    def shuffle(a,b):
        assert len(a) == len(b)
        ind = np.array([i for i in range(len(a))])
        random.shuffle(ind)
        return a[ind], b[ind]
    
    syn_data, syn_labels = shuffle(syn_data, syn_labels)
    
    index, syn_data_gen_noisy = flip_func(["gender", "x", "y"], syn_data, syn_labels, "gender", eta[0], eta[1])
    syn_data_noisy = copy.deepcopy(syn_data)
    syn_data_noisy[:, 0] = syn_data_gen_noisy
    
    return syn_data, syn_labels, syn_data_noisy, index, syn_data_test, syn_labels_test

def denoised_theta_with_perturbations(eta = [0, 0.05], tau=0.9, flip_func = flipping_far_from_boundary):
    #### initialize
    C=0
    lam=1/6 # accurate value of lambda on the distribution from which we draw the data
    delta=0.01
    rng_loc = rng
    np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
    
    H = np.array([[1-eta[0], eta[0]], [eta[1], 1-eta[1]]])
    
    acc = []; sr = []; fpr = []; tpr = []

    def select_job(job_id, ss):
        np.random.seed(job_id)
        if ss is not None: rng_loc = np.random.default_rng(ss)
        else: rng_loc = rng
        #
        #########################################
        #### load noisy data with imputed PA
        #########################################
        syn_data, syn_labels, syn_data_noisy, index,\
                     syn_data_test, syn_labels_test    = gen_data(eta, flip_func=flip_func)
        #
        denoised_theta = denoisedfair.denoised(syn_data_noisy, syn_labels, index, C, tau, H, "sr", lam=lam, delta=3*delta, in_use_prot_attr=False)
        if verbose: print('\n'*2)
        if verbose: print(f"THETA: {denoised_theta}")
        if verbose: print("test on true:", testing(syn_data, syn_data[:,index], syn_labels, index, denoised_theta))
        res = testing(syn_data_test, syn_data_test[:,index], syn_labels_test, index, denoised_theta)
            
            
        return res
        
    ss = rng.bit_generator._seed_seq ## seed sequence (source: https://albertcthomas.github.io/good-practices-random-number-generators/)
    child_states = ss.spawn(ITERS) ## child sequences
    answer = Parallel(n_jobs=CORES, verbose=100)(delayed(select_job)(int(i), child_states[i]) for i in range(ITERS))
    
    all_results = {}
    for i in range(ITERS):
        acc.append(answer[i]['acc'])
        sr.append(answer[i]['sr'])
        fpr.append(answer[i]['fpr'])
        tpr.append(answer[i]['tpr'])

    
    if verbose:
        print(f'\tacc\tsr\tfpr')
        print(f'Means:\t{np.round(np.mean(acc),3)}\t{np.round(np.mean(sr),3)}\t{np.round(np.mean(fpr),3)}')
        print(f'Std:\t{np.round(np.std(acc),3)}\t{np.round(np.std(sr),3)}\t{np.round(np.std(fpr),3)}')
        print('')
        print(f'Acc: {acc}')
        print(f'Sr: {sr}')
        print(f'Fpr: {fpr}')
        print(f'tpr: {tpr}')
    return acc, sr, fpr, tpr

def err_tolerant_theta_with_perturbations(eta = [0, 0.05], tau=0.9, flip_func = flipping_far_from_boundary):
    from scipy.optimize import minimize
    def computeWorstRate(a, b, c, d, eta=0):
        # (a-\eta_1)/(a+b-\eta_1+\eta_2) * (c+d+\eta_1-\eta_2)/(c+\eta_1)

        def obj(x):
            return (a-x[0]) * (c+d+x[0]-x[1]) / (a+b-x[0]+x[1]) / (c+x[0])

        def der(x):
            der0  = 0
            der0 += (a-x[0]) / (c+x[0]) / (a+b-x[0]+x[1])
            der0 += (a-x[0]) * (c+d+x[0]-x[1]) / (c+x[0]) / (a+b-x[0]+x[1])**2
            der0 -= (c+d+x[0]-x[1]) / (c+x[0]) / (a+b-x[0]+x[1])
            der0 -= (a-x[0]) * (c+d+x[0]-x[1]) / (c+x[0]) ** 2 / (a+b-x[0]+x[1])
            #
            der1  = 0
            der1 -= (a-x[0]) / (c+x[0]) / (a+b-x[0]+x[1])
            der1 -= (a-x[0]) * (c+d+x[0]-x[1]) / (c+x[0]) / (a+b-x[0]+x[1])**2
            #
            return np.array([der0, der1])

        def const(x):
            f = []
            f.append(eta - x[0] - x[1])
            f.append(eta - x[0])
            f.append(eta - x[1])
            f.append(c - x[0])
            f.append(b - x[1])
            f.append(x[0])
            f.append(x[1])
            return f

        res = {'success': False}
        mn = 1000

        for i in range(10):
            # initialize random solution
            x0 = np.random.rand(2)
            x0 *= eta / np.sum(x0)

            # initialize constraints
            ineq_cons = {'type': 'ineq', 'fun' : lambda x: const(x)}

            # solve problem
            res = minimize(fun = obj, x0 = x0, method='SLSQP', jac = der, constraints = [ineq_cons],\
                     options = {'maxiter': 100, 'ftol': 1e-6, 'eps' : 1e-6, 'disp': False})

            cst = const(res.x)

            if np.min(cst) < -1e-2:
                print(f"Solution violates the constraints!\nconstraints: {const}")
                continue

            mn = min(mn, obj(res.x) / obj(np.zeros(2)))

        return mn
    
    #########################################
    #### initialize
    #########################################
    C=0
    delta=0.01
    rng_loc = rng
    np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
    

    acc = []; sr = []; fpr = []; tpr = []

    for i in range(ITERS):
        #########################################
        #### load noisy data with imputed PA
        #########################################
        syn_data, syn_labels, syn_data_noisy, index,\
                     syn_data_test, syn_labels_test    = gen_data(eta, flip_func=flip_func)
        
        p0 = 0.5 
        p1 = 0.5 
        p00 = np.mean((syn_data_noisy[:, index] == 0) & (syn_labels == 0)) # 1/3
        p10 = np.mean((syn_data_noisy[:, index] == 1) & (syn_labels == 0)) # 1/6
        p01 = np.mean((syn_data_noisy[:, index] == 0) & (syn_labels == 1)) # 1/3
        p11 = np.mean((syn_data_noisy[:, index] == 1) & (syn_labels == 1)) # 1/6

        eta_avg = eta[0] * p0 + eta[1] * p1    
        tauRelaxed = tau * computeWorstRate(p01, p00, p11, p10, eta_avg)
        tauRelaxed = min(tauRelaxed, tau * computeWorstRate(p11, p10, p01, p00, eta_avg)) 
        lamRelaxed = min(p01, p11) - eta_avg - 3*delta
        
        print(f'lamRelaxed: {lamRelaxed}, tauRelaxed: {tauRelaxed}.')
        
        
        if verbose: print(f'Error-tolerant classifier for tau={tau}: ')
        err_tolerant_theta = denoisedfair.undenoised_lambda(syn_data_noisy, syn_labels, index, C, tauRelaxed, "sr", delta=0.00, lam=lamRelaxed)
        
        res = testing(syn_data_test, syn_data_test[:,index], syn_labels_test, index, err_tolerant_theta)
        
        acc.append(res['acc'])
        sr.append(res['sr'])
        fpr.append(res['fpr'])
        tpr.append(res['tpr'])
        if verbose: print('\n'*2)
    
    if verbose:
        print(f'\tacc\tsr\tfpr\t')
        print(f'Means:\t{np.round(np.mean(acc),3)}\t{np.round(np.mean(sr),3)}\t{np.round(np.mean(fpr),3)}\t{np.round(np.mean(tpr),3)}')
        print(f'Std:\t{np.round(np.std(acc),3)}\t{np.round(np.std(sr),3)}\t{np.round(np.std(fpr),3)}\t{np.round(np.std(tpr),3)}')
        print('')
        print(f'Acc: {acc}')
        print(f'Sr: {sr}')
        print(f'Fpr: {fpr}')
        print(f'Fpr: {tpr}')
    return acc, sr, fpr, tpr

### ErrTol

In [ ]:
def run_err_tol(tau=0.8):
    output = ''
    for e in [0.0,0.03,0.05]:
        acc, sr, fpr, tpr = err_tolerant_theta_with_perturbations(eta = [0, e], tau=tau,\
                                                         flip_func = flipping_far_from_boundary)

        output += f'Eta=[0,{e}]'
        output += '\n'
        output += f'\tacc\tsr\tfpr\ttpr'
        output += '\n'
        output += f'Means:\t{np.round(np.mean(acc),3)}\t{np.round(np.mean(sr),3)}\t{np.round(np.mean(fpr),3)}\t{np.round(np.mean(tpr),3)}'
        output += '\n'
        output += f'Std:\t{np.round(np.std(acc),3)}\t{np.round(np.std(sr),3)}\t{np.round(np.std(fpr),3)}\t{np.round(np.std(tpr),3)}'
        output += '\n'
        output += '\n'

    print('\n'*5)
    print(output)

In [ ]:
run_err_tol(tau=0.8)

### CHKV

In [ ]:
def run_denoised(tau=0.8):
    output = ''
    for e in tqdm([0.0,0.03,0.05]):
        acc, sr, fpr, tpr = denoised_theta_with_perturbations(eta = [e, 0], tau=tau,\
                                                         flip_func = flipping_far_from_boundary)

        output += f'Eta=[0,{e}]'
        output += '\n'
        output += f'\tacc\tsr\tfpr\ttpr'
        output += '\n'
        output += f'Means:\t{np.round(np.mean(acc),3)}\t{np.round(np.mean(sr),3)}\t{np.round(np.mean(fpr),3)}\t{np.round(np.mean(tpr),3)}'
        output += '\n'
        output += f'Std:\t{np.round(np.std(acc),3)}\t{np.round(np.std(sr),3)}\t{np.round(np.std(fpr),3)}\t{np.round(np.std(tpr),3)}'
        output += '\n'
        output += '\n'

    print('\n'*5)
    print(output)

In [ ]:
run_denoised(tau=0.8)

### Unconstrained

In [ ]:
run_denoised(tau=0.0)